In [1]:
#Homework 2 Question 2

#2(b) understanding the data

import json
import numpy as np
import pandas as pd

cuisines, trainIngredients, testIngredients, uniqueCuisines, uniqueIngredients = [], [], [], [], []

#loading json file
with open('train.json') as data_train:    
    trainData = json.load(data_train)

with open('test.json') as data_test:
    testData = json.load(data_test)

#constructing relevant arrays
for sample in trainData:
    cuisines.append(sample['cuisine'])
    trainIngredients.append(sample['ingredients'])
    for ingredient in sample['ingredients']:
        uniqueIngredients.append(ingredient)

for sample in testData:
    testIngredients.append(sample['ingredients'])
    
uniqueCuisines = np.unique(cuisines)
uniqueIngredients = np.unique(uniqueIngredients)

print ("there are %d samples, %d unique types of cuisines and %d unique types of ingredients in the training set" %(len(trainData), len(uniqueCuisines), len(uniqueIngredients)))


there are 39774 samples, 20 unique types of cuisines and 6714 unique types of ingredients in the training set


In [2]:
#2(c) constructing feature vectors for ingredients in each sample

#function to change boolean to binary 
#(http://stackoverflow.com/questions/16869990/how-to-convert-from-boolean-array-to-int-array-in-python)
def boolstr_to_floatstr(v):
    if v == 'True':
        return '1'
    elif v == 'False':
        return '0'
    else:
        return v

trainFeatures, testFeatures = [], [] 

#Creating feature vectors for dish ingredients
for sample in trainIngredients:
    trainFeatures.append(np.in1d(uniqueIngredients, sample))

trainFeatures = np.vectorize(boolstr_to_floatstr)(trainFeatures).astype(float)

for sample in testIngredients:
    testFeatures.append(np.in1d(uniqueIngredients,sample))

testFeatures = np.vectorize(boolstr_to_floatstr)(testFeatures).astype(float)

#creating data frame for feature vectors
dfTrain = pd.DataFrame(trainFeatures, columns = uniqueIngredients)
dfTest = pd.DataFrame(testFeatures, columns = uniqueIngredients)


In [4]:
#2(d) Average Classification Accuracy via Naïve Bayes Classifier

#http://scikit-learn.org/stable/modules/naive_bayes.html
#https://www.kaggle.com/wiki/GettingStartedWithPythonForDataScience/history/969

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn import cross_validation

gnb = GaussianNB()
bnb = BernoulliNB()

trainFeatures = np.array(trainFeatures)
cuisines = np.array(cuisines)

cv = cross_validation.KFold(len(trainFeatures), n_folds=3)

results_gaussian, results_bernoulli = [], []
for traincv, testcv in cv:
    results_gaussian.append(cross_validation.cross_val_score(gnb, trainFeatures[traincv], cuisines[traincv]))
    results_bernoulli.append(cross_validation.cross_val_score(bnb, trainFeatures[traincv], cuisines[traincv]))

print "Results for Gaussian Naive Bayes: " + str( np.array(results_gaussian).mean() )
print "Results for Bernoulli Naive Bayes: " + str( np.array(results_bernoulli).mean() )



Results for Gaussian Naive Bayes: 0.421795395376
Results for Bernoulli Naive Bayes: 0.645486431751
(array([13258, 13259, 13260, ..., 39771, 39772, 39773]), array([    0,     1,     2, ..., 13255, 13256, 13257]))
(array([    0,     1,     2, ..., 39771, 39772, 39773]), array([13258, 13259, 13260, ..., 26513, 26514, 26515]))
(array([    0,     1,     2, ..., 26513, 26514, 26515]), array([26516, 26517, 26518, ..., 39771, 39772, 39773]))


[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]




In [19]:
#2(f) Average Classification Accuracy via Logistic Regression

from sklearn.linear_model import LogisticRegression

log = LogisticRegression()

results_log = []

for traincv, testcv in cv:
    results_log.append(cross_validation.cross_val_score(log, trainFeatures[traincv], cuisines[traincv]))
    
print "Results for logistic regression: " + str( np.array(results_log).mean() )


Results for logistic regression: 0.76147862508


In [34]:
#2(f) Training - best performed classifier (logistic regression) and submitting results

columnName = ['id', 'cuisine']
testId, testCuisine  = [], []

for sample in testData:
    testId.append(sample['id'])
    testCuisine.append(0)

y_test_array = [testId, testCuisine]
y_test_array = np.array(y_test_array)
y_test = pd.DataFrame(y_test_array.T, columns = columnName)

log.fit(trainFeatures, cuisines)
gnb.fit(trainFeatures, cuisines)
bnb.fit(trainFeatures, cuisines)
predicted_log = log.predict(testFeatures)
predicted_gnb = gnb.predict(testFeatures)
predicted_bnb = bnb.predict(testFeatures)

y_test['cuisine'] = predicted_log
y_test.to_csv('cooking_logistic_regression.csv', index = False)

y_test['cuisine'] = predicted_gnb
y_test.to_csv('cooking_gaussian_naive_bayes.csv', index = False)

y_test['cuisine'] = predicted_bnb
y_test.to_csv('cooking_bernoulli_naive_bayes.csv', index = False)

#score for logistic, gnb and bnb are 0.78319, 0.34724 and 0.71148 respectively



In [9]:
print trainFeatures

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
